## Load required packages

In [23]:
import h5py
import numpy as np
import scanpy as sc
import torch
import random

from scAClc import run_scaclc


## Set seed

In [24]:
seed = 666
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)



## Read data

In [25]:
file_path = r'C:\Users\ROG\Downloads\human_pbmc.h5'
data_mat = h5py.File(file_path, 'r')
x, y = np.array(data_mat['X']), np.array(data_mat['Y'])
data_mat.close()

adata = sc.AnnData(x)
adata.obs['celltype'] = y

In [15]:
import h5py
import numpy as np
import scanpy as sc
from scipy.sparse import csr_matrix

def read_sparse_matrix(file_path):
    """
    从 HDF5 文件中读取稀疏矩阵数据并转换为密集矩阵
    """
    data_mat = h5py.File(file_path, 'r')
    data = np.array(data_mat['exprs']['data'])
    indices = np.array(data_mat['exprs']['indices'])
    indptr = np.array(data_mat['exprs']['indptr'])
    shape = tuple(np.array(data_mat['exprs']['shape']))
    sparse_matrix = csr_matrix((data, indices, indptr), shape=shape)
    data_mat.close()
    return sparse_matrix.toarray()

def read_labels(file_path):
    """
    从 HDF5 文件中读取标签信息
    """
    data_mat = h5py.File(file_path, 'r')
    obs = data_mat['obs']
    cell_name = np.array(obs['cell_type1'])
    cell_type, y = np.unique(cell_name, return_inverse=True)
    data_mat.close()
    return y

file_path = r'C:\Users\ROG\数据包\20 datasets\Muraro\data.h5'
#file_path = r'C:\Users\ROG\Downloads\Mouse_h.h5'
# 读取稀疏矩阵数据
x = read_sparse_matrix(file_path)

# 读取标签信息
y = read_labels(file_path)

# 显式指定数据类型
adata = sc.AnnData(x, dtype=x.dtype)

# 转换 adata.X 的数据类型为 float64
adata.X = adata.X.astype(np.float64)

# 确保标签长度和数据行数一致
if len(y) != adata.n_obs:
    print(f"标签长度 {len(y)} 与数据行数 {adata.n_obs} 不一致，请检查数据。")
else:
    adata.obs['celltype'] = y

print("成功创建 AnnData 对象，adata 基本信息：")
print(adata)

成功创建 AnnData 对象，adata 基本信息：
AnnData object with n_obs × n_vars = 2122 × 19046
    obs: 'celltype'


## Perform data pre-processing

In [26]:
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_cells(adata, min_genes=200)
adata.raw = adata.copy()

sc.pp.normalize_per_cell(adata)
adata.obs['scale_factor'] = adata.obs.n_counts / adata.obs.n_counts.mean()

sc.pp.log1p(adata)


In [27]:
adata

AnnData object with n_obs × n_vars = 4271 × 16486
    obs: 'celltype', 'n_genes', 'n_counts', 'scale_factor'
    var: 'n_cells'
    uns: 'log1p'

## Run scAClc

In default mode, function `run_scaclc` would return an AnnData object with embedding and clustering result stored in `adata.obsm['scaclc_emb']` and `adata.obs['scaclc_cluster']`

In [28]:
# adata = run_scaclc(adata)

If return_all is True, function `run_scaclc` would print and return all temporate results:

- K: Final number of clusters.
- pred_all: All temporary clustering results.
- emb_all: All temporary embedding.


In [29]:
# adata, K, pred_all, emb_all, run_time = run_scaclc(adata, return_all=True)

If the key of grount truth cell type (i.e. `cl_type`) is also provided, ARI and NMI will also be returned.

In [30]:
adata, nmi, ari, K, pred_all, emb_all, run_time= run_scaclc(adata, cl_type='celltype', return_all=True,seed=seed, 
    n_top_hvg=3000, 
    n_top_rfg=3000  )

>>> Feature selection (HVGs and RFGs) in progress...
>>> Identification of HVGs is currently in progress...
>>> Identified 3000 Highly Variable Genes.
>>> Identification of RFGs is currently in progress...
>>> Identified 3000 Random Forest based Genes.
>>> Total selected genes: 5265
>>> adata filtered to 5265 genes.
>>> adata.raw also filtered to 5265 genes.
Start pre-training! Total epochs is 200.


Pre-training::   0%|▎                                                               | 1/200 [00:01<04:40,  1.41s/epoch]

Pre-training epoch [1/200]. Average ZINB loss:0.8301, kld loss:8.5946, total loss:0.8387


Pre-training::   1%|▋                                                               | 2/200 [00:02<04:02,  1.22s/epoch]

Pre-training epoch [2/200]. Average ZINB loss:0.6364, kld loss:26.7377, total loss:0.6632


...

Train epoch [7/500]. ZINB loss:0.4702, kld loss:35.8965, cluster loss:0.3875, contrastive loss:5.6071, total loss:1.8854
Clustering   7: NMI= 0.8032, ARI= 0.7826, Delta=0.6776
Reach tolerance threshold. Perform cluster merging.
-----------------iter: 1-----------------
n_clusters: 7
count_true: 21
count: 21
Reach count!
Stop merging clusters! Continue updating several rounds.


Training::   2%|█                                                                   | 8/500 [00:12<13:03,  1.59s/epoch]

Train epoch [8/500]. ZINB loss:0.4705, kld loss:36.3404, cluster loss:0.3837, contrastive loss:5.6025, total loss:1.8852
Clustering   8: NMI= 0.7993, ARI= 0.7808, Delta=0.0089


Training::   2%|█▏                                                                  | 9/500 [00:14<13:03,  1.60s/epoch]

Train epoch [9/500]. ZINB loss:0.4703, kld loss:36.7565, cluster loss:0.3842, contrastive loss:5.6553, total loss:1.8953
Clustering   9: NMI= 0.7989, ARI= 0.7813, Delta=0.0061
Reach tolerance threshold. Stopping training.
Total time: 412.5707416534424 seconds


In [31]:
print("ARI = {}, NMI = {}".format(ari, nmi))

ARI = 0.78131, NMI = 0.79888
